# Twitter Retweet Network

In this tutorial, we will be creating a Twitter retweet network from a set of tweets. 

Note that public Twitter data sets that you will find will only contain the Tweet IDs (as per Twitter's policy), so you will have to "rehydrate" each tweet by using the Twitter API to request the full text. 

We provide two ways of visualizing the graph itself - one through iGraph and the other by exporting the networkx generated network into a Gephi file (note that the extension is deprecated by Gephi but is what networkx supports and still works).  

https://gephi.org/


In [ ]:
import json
import pickle 
import networkx as nx
import wget
import igraph as ig

# Save the location of the Tweet JSON file
TWEET_JSON = "covid.json"

In [ ]:
# create our tweet dictionary that filters out all the information we are not interested in
# a clean tweet object extracts and contains the following attributes from the raw tweet: 
# the original tweet's : id, user, text, created at
# if the tweet is a retweet, our object also contains the retweet's: rt_text, rt_user, rt_id
# along with a boolean value is_rt that indicates whether the current tweet is a retweet. 

def clean_tweet(tweet): 
    processed_tweet = {
        "id" : tweet['id'], 
        "user" : tweet['user']['screen_name'], 
        "created_at" : tweet['created_at'], 
        "text" : ""
    }
    if "extended_tweet" in tweet: 
        processed_tweet["text"] = tweet['extended_tweet']['full_text']
    elif "full_text" in tweet: 
        processed_tweet["text"] = tweet['full_text']
    elif "text" in tweet: 
        processed_tweet["text"] = tweet['text']
        
    if 'retweeted_status' in tweet:
        rt = tweet['retweeted_status']
        processed_tweet["is_rt"] = True
        processed_tweet["rt_user"] = rt['user']['screen_name']
        processed_tweet["rt_id"] = rt['id'] 
        processed_tweet["rt_text"] = ""
        if 'extended_tweet' in rt: 
            processed_tweet["rt_text"] = rt['extended_tweet']['full_text']
        elif "full_text" in rt:
            processed_tweet["rt_text"] = rt['full_text']
        elif "text" in rt: 
            processed_tweet["rt_text"] = rt['text']
    else: 
        processed_tweet["is_rt"] = False
            
    return processed_tweet
            
            
# load the raw tweets into a list
tweet_data = []
with open(TWEET_JSON) as f:
    for line in f:
        json_line = json.loads(line)
        tweet_data.append(json_line)

# filter all of the raw tweets by turning them into clean_tweet objects
# the filtering is taken care of in the class function
filtered_data = []
for elem in tweet_data: 
    filtered_tweet = clean_tweet(elem)
    filtered_data.append(filtered_tweet)

In [ ]:
# priting out the filtered_data, which contains clean_tweet objects 
filtered_data

In [ ]:
# printing the first element of the unfiltered data as an example
print(tweet_data[0])
print ("---")
# printing an example of a retweet object as an example
print(tweet_data[1])

In [ ]:
# printing the first element of the filtered data
print(filtered_data[0])
print ("---")
# printing an example of a retweet object
print(filtered_data[1])


In [ ]:
# how many total tweets we have
len(filtered_data)

In [ ]:
# construct a directed graph for retweet graph
G=nx.DiGraph()

for elem in filtered_data: 
    # for each tweet, we add the tweet's user to the graph
    G.add_node(elem["user"])
    print (elem["user"])

    # if this tweet is a retweet, we add the original tweet's user to the graph
    # and create an edge between the current user and the original tweet's user
    # current user -> original tweet's user
    if elem["is_rt"]: 
        G.add_node(elem["rt_user"])
        print (elem["rt_user"])
        G.add_edge(elem["user"], elem["rt_user"])
    print ("--")

In [ ]:
# printing out the number of nodes (users) in our graph
print (len(G.nodes()))
# printing out the number of edges/links in our graph
print (len(G.edges()))

In [ ]:
# writing our graph to a Gephi file format
nx.write_gexf(G, "covid.gexf")

In [ ]:
# Visualizing our retweet network
# This code was borrowed from the Plotly documentation
# https://plot.ly/python/v3/igraph-networkx-comparison/

G2 = ig.Graph.TupleList(G.edges(), directed=True)
labels=list(G2.vs['name'])
N = len(labels)
E = [e.tuple for e in G2.es]
layt=G2.layout('fr') #Fruchterman-Reingold force-directed layout

In [ ]:
# Below is code to visualize the graph. Treat as blackbox for now!
import chart_studio.plotly  as py
from plotly.graph_objs import *

Xn=[layt[k][0] for k in range(N)]
Yn=[layt[k][1] for k in range(N)]
Xe=[]
Ye=[]
for e in E:
    Xe+=[layt[e[0]][0],layt[e[1]][0], None]
    Ye+=[layt[e[0]][1],layt[e[1]][1], None]
    
trace1=Scatter(x=Xe,
               y=Ye,
               mode='lines',
               line= dict(color='rgb(210,210,210)', width=1),
               hoverinfo='none'
               )
trace2=Scatter(x=Xn,
               y=Yn,
               mode='markers',
               name='ntw',
               marker=dict(symbol='circle-dot',
                                        size=5,
                                        color='#6959CD',
                                        line=dict(color='rgb(50,50,50)', width=0.5)
                                        ),
               text=labels,
               hoverinfo='text'
               )

axis=dict(showline=False, # hide axis line, grid, ticklabels and  title
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title=''
          )

axis=dict(showline=False, # hide axis line, grid, ticklabels and  title
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title=''
          )

width=800
height=800
layout=Layout(title= "Twitter Retweet Network",
    font= dict(size=12),
    showlegend=False,
    autosize=False,
    width=width,
    height=height,
    xaxis=layout.XAxis(axis),
    yaxis=layout.YAxis(axis),
    margin=layout.Margin(
        l=40,
        r=40,
        b=85,
        t=100,
    ),
    hovermode='closest',
    annotations=[
           dict(
           showarrow=False,
            text='',
            xref='paper',
            yref='paper',
            x=0,
            y=-0.1,
            xanchor='left',
            yanchor='bottom',
            font=dict(
            size=14
            )
            )
        ]
    )

data=[trace1, trace2]
fig=Figure(data=data, layout=layout)

In [ ]:
fig.show()